In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
import xgboost as xgb


In [2]:
test=pd.read_csv("test.csv")
df=pd.read_csv("train.csv")

In [3]:
df.head()

,timestamp,value,is_anomaly,predicted
0,1425008573,42,False,44.072500
1,1425008873,41,False,50.709390
2,1425009173,41,False,81.405120
3,1425009473,61,False,39.950367
4,1425009773,44,False,35.350160


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15830 entries, 0 to 15829
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   timestamp   15830 non-null  int64  
 1   value       15830 non-null  int64  
 2   is_anomaly  15830 non-null  bool   
 3   predicted   15830 non-null  float64
dtypes: bool(1), float64(1), int64(2)
memory usage: 386.6 KB


In [5]:
df.describe

<bound method NDFrame.describe of         timestamp  value  is_anomaly  predicted
0      1425008573     42       False  44.072500
1      1425008873     41       False  50.709390
2      1425009173     41       False  81.405120
3      1425009473     61       False  39.950367
4      1425009773     44       False  35.350160
...           ...    ...         ...        ...
15825  1429756073     44       False  53.624115
15826  1429756373     45       False  59.752296
15827  1429756673     48       False  52.147630
15828  1429756973     26       False  58.007545
15829  1429757273     38       False  59.144700

[15830 rows x 4 columns]>

In [6]:
df.duplicated().sum()

0

In [7]:
df.isna().sum()

timestamp     0
value         0
is_anomaly    0
predicted     0
dtype: int64

In [8]:
df['is_anomaly'].value_counts()

False    15054
True       776
Name: is_anomaly, dtype: int64

In [9]:
df['value']-df['predicted']

0        -2.072500
1        -9.709390
2       -40.405120
3        21.049633
4         8.649840
           ...    
15825    -9.624115
15826   -14.752296
15827    -4.147630
15828   -32.007545
15829   -21.144700
Length: 15830, dtype: float64

In [10]:
df.corr()

,timestamp,value,is_anomaly,predicted
timestamp,1.000000,0.032628,0.016457,0.030462
value,0.032628,1.000000,0.324859,0.445180
is_anomaly,0.016457,0.324859,1.000000,0.059719
predicted,0.030462,0.445180,0.059719,1.000000


In [11]:
X = df[['timestamp', 'value', 'predicted']]
y = df['is_anomaly']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
svm_model = SVC(kernel='poly', C=1.0, gamma='scale')

svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report(y_test, y_pred, zero_division=1))

Accuracy: 0.9466203411244473
Classification Report:
               precision    recall  f1-score   support

       False       0.95      1.00      0.97      2997
        True       1.00      0.00      0.00       169

    accuracy                           0.95      3166
   macro avg       0.97      0.50      0.49      3166
weighted avg       0.95      0.95      0.92      3166



In [13]:
lg_model = LogisticRegression(solver='liblinear', max_iter=1000)
lg_model.fit(X_train, y_train)
lg_ypred = lg_model.predict(X_test)
print("Logistic Regression:")
print(f'Accuracy: {accuracy_score(y_test, lg_ypred)}')
print("Classification Report:\n", classification_report(y_test, lg_ypred, zero_division=1))

Logistic Regression:
Accuracy: 0.9466203411244473
Classification Report:
               precision    recall  f1-score   support

       False       0.95      1.00      0.97      2997
        True       1.00      0.00      0.00       169

    accuracy                           0.95      3166
   macro avg       0.97      0.50      0.49      3166
weighted avg       0.95      0.95      0.92      3166



In [14]:
rfc_model = RandomForestClassifier()
rfc_model.fit(X_train, y_train)
rfc_ypred = rfc_model.predict(X_test)
print("\nRandom Forest Classifier:")
print(f'Accuracy: {accuracy_score(y_test, rfc_ypred)}')
print("Classification Report:\n", classification_report(y_test, rfc_ypred, zero_division=1))


Random Forest Classifier:
Accuracy: 0.9718888186986734
Classification Report:
               precision    recall  f1-score   support

       False       0.98      0.99      0.99      2997
        True       0.82      0.60      0.70       169

    accuracy                           0.97      3166
   macro avg       0.90      0.80      0.84      3166
weighted avg       0.97      0.97      0.97      3166



In [15]:
xg = xgb.XGBClassifier(
    objective='binary:logistic',  
    eval_metric='logloss',             
    n_estimators=100,             
    learning_rate=0.1,            
    max_depth=3  
)
xg.fit(X_train, y_train)
xg_pred = xg.predict(X_test)
accuracy = accuracy_score(y_test, xg_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report(y_test, xg_pred, zero_division=1))



Accuracy: 0.9715729627289956
Classification Report:
               precision    recall  f1-score   support

       False       0.98      0.99      0.99      2997
        True       0.86      0.56      0.68       169

    accuracy                           0.97      3166
   macro avg       0.92      0.78      0.83      3166
weighted avg       0.97      0.97      0.97      3166



## RESULTS before oversampling:

### 1. SVC - 94.66 %
### 2. Random Forest - 97.12 %
### 3. XXGBOOST - 97.15 %
### 4. Logistic Regression - 94.6 %

In [16]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
print("Class distribution before SMOTE:", y_train.value_counts())
print("Class distribution after SMOTE:", y_train_resampled.value_counts())


Class distribution before SMOTE: False    12057
True       607
Name: is_anomaly, dtype: int64
Class distribution after SMOTE: False    12057
True     12057
Name: is_anomaly, dtype: int64


In [17]:
svm_model = SVC(kernel='poly', C=1.0, gamma='scale')

svm_model.fit(X_train_resampled, y_train_resampled)
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report(y_test, y_pred, zero_division=1))

Accuracy: 0.5022109917877448
Classification Report:
               precision    recall  f1-score   support

       False       0.95      0.50      0.66      2997
        True       0.06      0.55      0.11       169

    accuracy                           0.50      3166
   macro avg       0.51      0.52      0.38      3166
weighted avg       0.90      0.50      0.63      3166



In [18]:
lg_model = LogisticRegression(solver='liblinear', max_iter=1000)
lg_model.fit(X_train_resampled, y_train_resampled)
lg_ypred = lg_model.predict(X_test)
print("Logistic Regression:")
print(f'Accuracy: {accuracy_score(y_test, lg_ypred)}')
print("Classification Report:\n", classification_report(y_test, lg_ypred, zero_division=1))

Logistic Regression:
Accuracy: 0.9478837650031585
Classification Report:
               precision    recall  f1-score   support

       False       0.99      0.96      0.97      2997
        True       0.51      0.75      0.60       169

    accuracy                           0.95      3166
   macro avg       0.75      0.85      0.79      3166
weighted avg       0.96      0.95      0.95      3166



In [19]:
rfc_model = RandomForestClassifier()
rfc_model.fit(X_train_resampled, y_train_resampled)
rfc_ypred = rfc_model.predict(X_test)
print("\nRandom Forest Classifier:")
print(f'Accuracy: {accuracy_score(y_test, rfc_ypred)}')
print("Classification Report:\n", classification_report(y_test, rfc_ypred, zero_division=1))


Random Forest Classifier:
Accuracy: 0.9595704358812381
Classification Report:
               precision    recall  f1-score   support

       False       0.99      0.97      0.98      2997
        True       0.59      0.83      0.69       169

    accuracy                           0.96      3166
   macro avg       0.79      0.90      0.83      3166
weighted avg       0.97      0.96      0.96      3166



In [20]:
xg = xgb.XGBClassifier(
    objective='binary:logistic',  
    eval_metric='logloss',             
    n_estimators=100,             
    learning_rate=0.1,            
    max_depth=3  
)
xg.fit(X_train_resampled, y_train_resampled)
xg_pred = xg.predict(X_test)
accuracy = accuracy_score(y_test, xg_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", classification_report(y_test, xg_pred, zero_division=1))



Accuracy: 0.9456727732154138
Classification Report:
               precision    recall  f1-score   support

       False       0.99      0.95      0.97      2997
        True       0.49      0.87      0.63       169

    accuracy                           0.95      3166
   macro avg       0.74      0.91      0.80      3166
weighted avg       0.97      0.95      0.95      3166



## RESULTS after oversampling:

### 1. SVC - 50.02 %
### 2. Random Forest - 95.95 %
### 3. XXGBOOST - 94.56 %
### 4. Logistic Regression - 94.7 %